# Python Client for GCS

Google Cloud Storage (GCS), bucket to hold all your information.

This notebook illustrates some common interaction with GCS and provides tips.

Resources:
- [Product](https://cloud.google.com/storage)
- [Client API](https://github.com/googleapis/python-storage)
- [Client API Documentation](https://cloud.google.com/python/docs/reference/storage/latest)


---
## Setup

inputs:

In [2]:
EXPERIMENT = 'gcs'

packages:

In [3]:
from google.cloud import storage

clients:

In [4]:
gcs = storage.Client()

parameters:

In [5]:
DIR = f'temp/{EXPERIMENT}'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}